# Working with Datastores

Although it's fairly common for data scientists to work with data on their local file system, in an enterprise environment it can be more effective to store the data in a central location where multiple data scientists can access it. In this lab, you'll store data in the cloud, and use an Azure Machine Learning *datastore* to access it.

> **Important**: The code in this notebooks assumes that you have completed the first two tasks in Lab 4A. If you have not done so, go and do it now!


## Connect to Your Workspace

To access your datastore using the Azure Machine Learning SDK, you need to connect to your workspace.

> **Note**: If the authenticated session with your Azure subscription has expired since you completed the previous exercise, you'll be prompted to reauthenticate.

In [5]:
import azureml.core
from azureml.core import Workspace

# Load the workspace from the saved config file
ws = Workspace.from_config()
print('Ready to use Azure ML {} to work with {}'.format(azureml.core.VERSION, ws.name))

Ready to use Azure ML 1.13.0 to work with BSM_MLWorkspace3


## View Datastores in the Workspace

The workspace contains several datastores, including the **aml_data** datastore you ceated in the [previous task](labdocs/Lab04A.md).

Run the following code to retrieve the *default* datastore, and then list all of the datastores indicating which is the default.

In [6]:
from azureml.core import Datastore

# Get the default datastore
default_ds = ws.get_default_datastore()

# Enumerate all datastores, indicating which is the default
for ds_name in ws.datastores:
    print(ds_name, "- Default =", ds_name == default_ds.name)

aml_data - Default = False
workspaceblobstore - Default = True
workspacefilestore - Default = False


## Get a Datastore to Work With

You want to work with the **aml_data** datastore, so you need to get it by name:

In [7]:
aml_datastore = Datastore.get(ws, 'aml_data')
print(aml_datastore.name,":", aml_datastore.datastore_type + " (" + aml_datastore.account_name + ")")

aml_data : AzureBlob (bsmstorageaccount1)


## Set the Default Datastore

You are primarily going towork with the **aml_data** datastore in this course; so for convenience, you can set it to be the default datastore:

In [8]:
ws.set_default_datastore('aml_data')
default_ds = ws.get_default_datastore()
print(default_ds.name)

aml_data


## Upload Data to a Datastore

Now that you have identified the datastore you want to work with, you can upload files from your local file system so that they will be accessible to experiments running in the workspace, regardless of where the experiment script is actually being run.

In [9]:
default_ds.upload_files(files=['./data/diabetes.csv', './data/diabetes2.csv'], # Upload the diabetes csv files in /data
                       target_path='diabetes-data/', # Put it in a folder path in the datastore
                       overwrite=True, # Replace existing files of the same name
                       show_progress=True)

Uploading an estimated of 2 files
Uploading ./data/diabetes.csv
Uploaded ./data/diabetes.csv, 1 files out of an estimated total of 2
Uploading ./data/diabetes2.csv
Uploaded ./data/diabetes2.csv, 2 files out of an estimated total of 2
Uploaded 2 files


$AZUREML_DATAREFERENCE_68dd615236a34bcfbd073866a2d94d67

## Train a Model from a Datastore

When you uploaded the files in the code cell above, note that the code returned a *data reference*. A data reference provides a way to pass the path to a folder in a datastore to a script, regardless of where the script is being run, so that the script can access data in the datastore location.

The following code gets a reference to the **diabetes-data** folder where you uploaded the diabetes CSV files, and specifically configures the data reference for *download* - in other words, it can be used to download the contents of the folder to the compute context where the data reference is being used. Downloading data works well for small volumes of data that will be processed on local compute. When working with remote compute, you can also configure a data reference to *mount* the datastore location and read data directly from the data source.

> **More Information**: For more details about using datastores, see the [Azure ML documentation](https://docs.microsoft.com/azure/machine-learning/how-to-access-data).

In [10]:
data_ref = default_ds.path('diabetes-data').as_download(path_on_compute='diabetes_data')
print(data_ref)

$AZUREML_DATAREFERENCE_2fab95bd398a4b1a81bde24fc987bed2


To use the data reference in a training script, you must define a parameter for it. Run the following two code cells to create:

1. A folder named **diabetes_training_from_datastore**
2. A script that trains a classification model by using the training data in all of the CSV files in the folder referenced by the data reference parameter passed to it.

In [11]:
import os

# Create a folder for the experiment files
experiment_folder = 'diabetes_training_from_datastore'
os.makedirs(experiment_folder, exist_ok=True)
print(experiment_folder, 'folder created.')

diabetes_training_from_datastore folder created.


In [12]:
%%writefile $experiment_folder/diabetes_training.py
# Import libraries
import os
import argparse
from azureml.core import Run
import pandas as pd
import numpy as np
import joblib
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve

# Get parameters
parser = argparse.ArgumentParser()
parser.add_argument('--regularization', type=float, dest='reg_rate', default=0.01, help='regularization rate')
parser.add_argument('--data-folder', type=str, dest='data_folder', help='data folder reference')
args = parser.parse_args()
reg = args.reg_rate

# Get the experiment run context
run = Run.get_context()

# load the diabetes data from the data reference
data_folder = args.data_folder
print("Loading data from", data_folder)
# Load all files and concatenate their contents as a single dataframe
all_files = os.listdir(data_folder)
diabetes = pd.concat((pd.read_csv(os.path.join(data_folder,csv_file)) for csv_file in all_files))

# Separate features and labels
X, y = diabetes[['Pregnancies','PlasmaGlucose','DiastolicBloodPressure','TricepsThickness','SerumInsulin','BMI','DiabetesPedigree','Age']].values, diabetes['Diabetic'].values

# Split data into training set and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=0)

# Train a logistic regression model
print('Training a logistic regression model with regularization rate of', reg)
run.log('Regularization Rate',  np.float(reg))
model = LogisticRegression(C=1/reg, solver="liblinear").fit(X_train, y_train)

# calculate accuracy
y_hat = model.predict(X_test)
acc = np.average(y_hat == y_test)
print('Accuracy:', acc)
run.log('Accuracy', np.float(acc))

# calculate AUC
y_scores = model.predict_proba(X_test)
auc = roc_auc_score(y_test,y_scores[:,1])
print('AUC: ' + str(auc))
run.log('AUC', np.float(auc))

os.makedirs('outputs', exist_ok=True)
# note file saved in the outputs folder is automatically uploaded into experiment record
joblib.dump(value=model, filename='outputs/diabetes_model.pkl')

run.complete()

Writing diabetes_training_from_datastore/diabetes_training.py


The script will load the training data from the data reference passed to it as a parameter, so now you just need to set up the script parameters to pass the file reference when we run the experiment.

In [13]:
from azureml.train.estimator import Estimator
from azureml.core import Experiment, Environment
from azureml.widgets import RunDetails

# Create a Python environment
env = Environment("env")
env.python.user_managed_dependencies = True
env.docker.enabled = False

# Set up the parameters
script_params = {
    '--regularization': 0.1, # regularization rate
    '--data-folder': data_ref # data reference to download files from datastore
}


# Create an estimator
estimator = Estimator(source_directory=experiment_folder,
                      entry_script='diabetes_training.py',
                      script_params=script_params,
                      compute_target = 'local',
                      environment_definition=env
                   )

# Create an experiment
experiment_name = 'diabetes-training'
experiment = Experiment(workspace=ws, name=experiment_name)

# Run the experiment
run = experiment.submit(config=estimator)
# Show the run details while running
RunDetails(run).show()
run.wait_for_completion()

WARNING - If 'script' has been provided here and a script file name has been specified in 'run_config', 'script' provided in ScriptRunConfig initialization will take precedence.
WARNING - If 'arguments' has been provided here and arguments have been specified in 'run_config', 'arguments' provided in ScriptRunConfig initialization will take precedence.


_UserRunWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', '…

{'runId': 'diabetes-training_1598961055_0bb52a67',
 'target': 'local',
 'status': 'Finalizing',
 'startTimeUtc': '2020-09-01T11:50:57.845117Z',
 'properties': {'_azureml.ComputeTargetType': 'local',
  'ContentSnapshotId': '09f5f5f0-e09e-4339-9d9e-c1b51b8a85bd',
  'azureml.git.repository_uri': 'https://github.com/MicrosoftLearning/DP100',
  'mlflow.source.git.repoURL': 'https://github.com/MicrosoftLearning/DP100',
  'azureml.git.branch': 'master',
  'mlflow.source.git.branch': 'master',
  'azureml.git.commit': 'b3ab0f6abbc766abee8a6cb1e300cd21f80fef96',
  'mlflow.source.git.commit': 'b3ab0f6abbc766abee8a6cb1e300cd21f80fef96',
  'azureml.git.dirty': 'True'},
 'inputDatasets': [],
 'outputDatasets': [],
 'runDefinition': {'script': 'diabetes_training.py',
  'scriptType': None,
  'useAbsolutePath': False,
  'arguments': ['--regularization',
   '0.1',
   '--data-folder',
   '$AZUREML_DATAREFERENCE_2fab95bd398a4b1a81bde24fc987bed2'],
  'sourceDirectoryDataStore': None,
  'framework': 'Python

The first time the experiment is run, it may take some time to set up the Python environment - subsequent runs will be quicker.

When the experiment has completed, in the widget, view the output log to verify that the data files were downloaded.

As with all experiments, you can view the details of the experiment run in [Azure ML studio](https://ml.azure.com), and you can write code to retrieve the metrics and files generated:

In [14]:
# Get logged metrics
metrics = run.get_metrics()
for key in metrics.keys():
        print(key, metrics.get(key))
print('\n')
for file in run.get_file_names():
    print(file)

Regularization Rate 0.1
Accuracy 0.7893333333333333
AUC 0.8568655044545174


azureml-logs/60_control_log.txt
azureml-logs/70_driver_log.txt
logs/azureml/59615_azureml.log
logs/azureml/dataprep/python_span_8e090cb5-0a7d-42dc-8357-04e4f2e23ef6.jsonl
outputs/diabetes_model.pkl


Once again, you can register the model that was trained by the experiment.

In [15]:
from azureml.core import Model

# Register the model
run.register_model(model_path='outputs/diabetes_model.pkl', model_name='diabetes_model',
                   tags={'Training context':'Using Datastore'}, properties={'AUC': run.get_metrics()['AUC'], 'Accuracy': run.get_metrics()['Accuracy']})

# List the registered models
print("Registered Models:")
for model in Model.list(ws):
    print(model.name, 'version:', model.version)
    for tag_name in model.tags:
        tag = model.tags[tag_name]
        print ('\t',tag_name, ':', tag)
    for prop_name in model.properties:
        prop = model.properties[prop_name]
        print ('\t',prop_name, ':', prop)
    print('\n')

Registered Models:
diabetes_model version: 3
	 Training context : Using Datastore
	 AUC : 0.8568655044545174
	 Accuracy : 0.7893333333333333


diabetes_model version: 2
	 Training context : Parameterized SKLearn Estimator
	 AUC : 0.8483904671874223
	 Accuracy : 0.7736666666666666


diabetes_model version: 1
	 Training context : Estimator
	 AUC : 0.8484929598487486
	 Accuracy : 0.774




In this exercise, you've explored some options for working with data in the form of *datastores*.

Azure Machine Learning offers a further level of abstraction for data in the form of *datasets*, which you'll explore next.